### Overview of the dataset (API)

In [1]:
### imports the library requests :
import requests

### making a request to API OpenSky Network and check its status :
response = requests.get('https://opensky-network.org/api/flights/all?begin=1539212340&end=1539215940')
print(response.status_code)

200


###### The given time interval must not be larger than 30 days!
###### 200 : everything is OK and the API is ready to be retreived.

### Flights by time interval

###### 1- Visualisation of our data as dataframe

In [2]:
import pandas as pd

df=pd.read_json('https://opensky-network.org/api/flights/all?begin=1539212340&end=1539215940')
df.head()

,arrivalAirportCandidatesCount,callsign,departureAirportCandidatesCount,estArrivalAirport,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,estDepartureAirport,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,firstSeen,icao24,lastSeen
0,1,N344A,1,K0L7,24509.0,1118.0,KHND,2123.0,185.0,1539213540,a3cc2a,1539214941
1,1,FY1322,0,WMSA,7016.0,300.0,None,NaN,NaN,1539213131,750278,1539214628
2,8,N6750H,1,WT77,6694.0,722.0,00WA,12319.0,1478.0,1539213183,a8f0ab,1539213855
3,7,N846CD,1,KC83,6081.0,1682.0,KRHV,382.0,19.0,1539213707,ab9659,1539214467
4,1,N9863L,1,KCCR,312.0,22.0,KOAK,1257.0,4.0,1539213890,adc43a,1539214647


###### We retreived the API of all flights between 10-10-2018 23:59:00 until 11-10-2018 00:59:00 

###### 2- Dimensions of our data 

In [3]:
print('Dataframe dimensions:', df.shape)

Dataframe dimensions: (706, 12)


###### the number of rows is not fixed 

###### 3- Types of columns in our dataframe :

In [4]:
# gives some infos on columns types and number of null values

print(df.dtypes)

tab_info=pd.DataFrame(df.dtypes).T.rename(index={0:'column type'})
tab_info=tab_info.append(pd.DataFrame(df.isnull().sum()).T.rename(index={0:'null values (nb)'}))
tab_info

arrivalAirportCandidatesCount         int64
callsign                             object
departureAirportCandidatesCount       int64
estArrivalAirport                    object
estArrivalAirportHorizDistance      float64
estArrivalAirportVertDistance       float64
estDepartureAirport                  object
estDepartureAirportHorizDistance    float64
estDepartureAirportVertDistance     float64
firstSeen                             int64
icao24                               object
lastSeen                              int64
dtype: object


,arrivalAirportCandidatesCount,callsign,departureAirportCandidatesCount,estArrivalAirport,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,estDepartureAirport,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,firstSeen,icao24,lastSeen
column type,int64,object,int64,object,float64,float64,object,float64,float64,int64,object,int64
null values (nb),0,42,0,260,260,260,296,296,296,0,0,0


### Definitions of the variables :

#### Each entry in the API corresponds to a flight and we see that 526 flights have been recorded in one hour on October 10th 2018 between 8am and 9am. 
#### These flights are described according to 12 variables. A description of these variables can be found here and I briefly recall the meaning of the variables that will be used in this notebook:

#### 1) arrivalAirportCandidatesCount : Number of other possible departure airports. These are airports in short distance to estArrivalAirport.
#### 2) callsign : Callsign of the vehicle (8 chars). Can be null if no callsign has been received. If the vehicle transmits multiple callsigns during the flight, we take the one seen most frequently. a unique designation for a transmitter station.
#### 3) departureAirportCandidatesCount : Number of other possible departure airports. These are airports in short distance to estDepartureAirport.
#### 4) estArrivalAirport : ICAO code of the estimated arrival airport. Can be null if the airport could not be identified.
#### 5) estArrivalAirportHorizDistance : Horizontal distance of the last received airborne position to the estimated arrival airport in meters.
#### 6) estArrivalAirportVertDistance : Vertical distance of the last received airborne position to the estimated arrival airport in meters.
#### 7) estDepartureAirport : ICAO code of the estimated departure airport. Can be null if the airport could not be identified.
#### 8) estDepartureAirportHorizDistance : Horizontal distance of the last received airborne position to the estimated departure airport in meters.
#### 9) estDepartureAirportVertDistance : Vertical distance of the last received airborne position to the estimated departure airport in meters.
#### 10) firstSeen : Estimated time of departure for the flight as Unix time (seconds since epoch).
#### 11) icao24 : Unique ICAO 24-bit address of the transponder in hex string representation. All letters are lower case.  International Civil Aviation Organisation
#### 12) lastSeen : Estimated time of arrival for the flight as Unix time (seconds since epoch).

### Cleaning the data 

###### 1- Delete the variables we won't use 

In [5]:
var_removed = ['arrivalAirportCandidatesCount', 'callsign', 'departureAirportCandidatesCount']
df2 = df.drop(var_removed, axis=1)
df2.head() # the new dataframe after removing the variables not needed 

,estArrivalAirport,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,estDepartureAirport,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,firstSeen,icao24,lastSeen
0,K0L7,24509.0,1118.0,KHND,2123.0,185.0,1539213540,a3cc2a,1539214941
1,WMSA,7016.0,300.0,None,NaN,NaN,1539213131,750278,1539214628
2,WT77,6694.0,722.0,00WA,12319.0,1478.0,1539213183,a8f0ab,1539213855
3,KC83,6081.0,1682.0,KRHV,382.0,19.0,1539213707,ab9659,1539214467
4,KCCR,312.0,22.0,KOAK,1257.0,4.0,1539213890,adc43a,1539214647


In [6]:
# dimensions of our new data :

print('Dataframe dimensions:', df2.shape)

Dataframe dimensions: (706, 9)


###### 2- Missing values in our dataframe :

In [7]:
# Missing values in our dataframe :

missing_df2 = df2.isnull().sum(axis=0).reset_index()
missing_df2.columns = ['variable', 'missing values']
missing_df2

,variable,missing values
0,estArrivalAirport,260
1,estArrivalAirportHorizDistance,260
2,estArrivalAirportVertDistance,260
3,estDepartureAirport,296
4,estDepartureAirportHorizDistance,296
5,estDepartureAirportVertDistance,296
6,firstSeen,0
7,icao24,0
8,lastSeen,0


In [8]:
# dropping the null values :

data = df2.dropna()
data.head() # our new dataset 
data.shape

(237, 9)

In [9]:
# check if everything was done correctly :

missing_data = data.isnull().sum(axis=0).reset_index()
missing_data.columns = ['variable', 'missing values']
missing_data

,variable,missing values
0,estArrivalAirport,0
1,estArrivalAirportHorizDistance,0
2,estArrivalAirportVertDistance,0
3,estDepartureAirport,0
4,estDepartureAirportHorizDistance,0
5,estDepartureAirportVertDistance,0
6,firstSeen,0
7,icao24,0
8,lastSeen,0


###### 3- Changing the type of some variables 'firstSeen' and 'lastSeen' to datetime formats :

In [10]:
# changing the variable'firstSeen' to datetime formats :

from datetime import datetime

for index, row in data.iterrows():   
    dt_object1 = datetime.fromtimestamp(row['firstSeen'])
    print("dt_object1 =", dt_object1)  

dt_object1 = 2018-10-11 00:19:00
dt_object1 = 2018-10-11 00:13:03
dt_object1 = 2018-10-11 00:21:47
dt_object1 = 2018-10-11 00:24:50
dt_object1 = 2018-10-11 00:05:18
dt_object1 = 2018-10-10 23:59:48
dt_object1 = 2018-10-11 00:12:02
dt_object1 = 2018-10-11 00:02:37
dt_object1 = 2018-10-11 00:29:14
dt_object1 = 2018-10-11 00:04:36
dt_object1 = 2018-10-11 00:08:08
dt_object1 = 2018-10-11 00:17:45
dt_object1 = 2018-10-11 00:24:32
dt_object1 = 2018-10-11 00:00:09
dt_object1 = 2018-10-11 00:12:32
dt_object1 = 2018-10-11 00:07:20
dt_object1 = 2018-10-11 00:25:00
dt_object1 = 2018-10-11 00:14:56
dt_object1 = 2018-10-11 00:02:18
dt_object1 = 2018-10-11 00:00:15
dt_object1 = 2018-10-11 00:14:14
dt_object1 = 2018-10-11 00:09:15
dt_object1 = 2018-10-11 00:21:19
dt_object1 = 2018-10-11 00:19:57
dt_object1 = 2018-10-11 00:05:10
dt_object1 = 2018-10-11 00:34:32
dt_object1 = 2018-10-11 00:06:06
dt_object1 = 2018-10-11 00:10:31
dt_object1 = 2018-10-11 00:26:48
dt_object1 = 2018-10-11 00:28:00
dt_object1

In [11]:
# adding a new column to our data containing the new values of 'firstSeen' :

from datetime import datetime

for index, row in data.iterrows():
    data['firstSeenNew'] = data.apply(lambda row: datetime.fromtimestamp(row['firstSeen']), axis=1) 

/home/amira/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [12]:
# checking that the column was added :
data.head()

,estArrivalAirport,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,estDepartureAirport,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,firstSeen,icao24,lastSeen,firstSeenNew
0,K0L7,24509.0,1118.0,KHND,2123.0,185.0,1539213540,a3cc2a,1539214941,2018-10-11 00:19:00
2,WT77,6694.0,722.0,00WA,12319.0,1478.0,1539213183,a8f0ab,1539213855,2018-10-11 00:13:03
3,KC83,6081.0,1682.0,KRHV,382.0,19.0,1539213707,ab9659,1539214467,2018-10-11 00:21:47
4,KCCR,312.0,22.0,KOAK,1257.0,4.0,1539213890,adc43a,1539214647,2018-10-11 00:24:50
5,KDCA,5362.0,254.0,KEWR,385.0,13.0,1539212718,a9ffeb,1539215231,2018-10-11 00:05:18


In [13]:
# changing the variable'lastSeen' to datetime formats :

for index, row in data.iterrows():
    dt_object2 = datetime.fromtimestamp(row['lastSeen'])
    print("dt_object2 =", dt_object2)

dt_object2 = 2018-10-11 00:42:21
dt_object2 = 2018-10-11 00:24:15
dt_object2 = 2018-10-11 00:34:27
dt_object2 = 2018-10-11 00:37:27
dt_object2 = 2018-10-11 00:47:11
dt_object2 = 2018-10-11 00:29:35
dt_object2 = 2018-10-11 00:48:26
dt_object2 = 2018-10-11 00:17:20
dt_object2 = 2018-10-11 00:49:48
dt_object2 = 2018-10-11 00:22:50
dt_object2 = 2018-10-11 00:36:28
dt_object2 = 2018-10-11 00:36:15
dt_object2 = 2018-10-11 00:48:21
dt_object2 = 2018-10-11 00:21:08
dt_object2 = 2018-10-11 00:39:23
dt_object2 = 2018-10-11 00:29:56
dt_object2 = 2018-10-11 00:45:02
dt_object2 = 2018-10-11 00:42:46
dt_object2 = 2018-10-11 00:12:29
dt_object2 = 2018-10-11 00:31:19
dt_object2 = 2018-10-11 00:31:46
dt_object2 = 2018-10-11 00:23:29
dt_object2 = 2018-10-11 00:35:36
dt_object2 = 2018-10-11 00:44:38
dt_object2 = 2018-10-11 00:43:50
dt_object2 = 2018-10-11 00:49:56
dt_object2 = 2018-10-11 00:38:27
dt_object2 = 2018-10-11 00:38:12
dt_object2 = 2018-10-11 00:44:27
dt_object2 = 2018-10-11 00:42:20
dt_object2

In [14]:
# adding a new column to our data containing the new values of 'lastSeen' :

for index, row in data.iterrows():
    data['lastSeenNew'] = data.apply(lambda row: datetime.fromtimestamp(row['lastSeen']), axis=1)

/home/amira/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [15]:
# checking that the column was added :
data.head()

,estArrivalAirport,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,estDepartureAirport,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,firstSeen,icao24,lastSeen,firstSeenNew,lastSeenNew
0,K0L7,24509.0,1118.0,KHND,2123.0,185.0,1539213540,a3cc2a,1539214941,2018-10-11 00:19:00,2018-10-11 00:42:21
2,WT77,6694.0,722.0,00WA,12319.0,1478.0,1539213183,a8f0ab,1539213855,2018-10-11 00:13:03,2018-10-11 00:24:15
3,KC83,6081.0,1682.0,KRHV,382.0,19.0,1539213707,ab9659,1539214467,2018-10-11 00:21:47,2018-10-11 00:34:27
4,KCCR,312.0,22.0,KOAK,1257.0,4.0,1539213890,adc43a,1539214647,2018-10-11 00:24:50,2018-10-11 00:37:27
5,KDCA,5362.0,254.0,KEWR,385.0,13.0,1539212718,a9ffeb,1539215231,2018-10-11 00:05:18,2018-10-11 00:47:11


In [16]:
# deleting the old columns :

var_removed = ['firstSeen', 'lastSeen']
NewData = data.drop(var_removed, axis=1)
NewData.head() # the new dataframe after removing the variables not needed 

,estArrivalAirport,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,estDepartureAirport,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,icao24,firstSeenNew,lastSeenNew
0,K0L7,24509.0,1118.0,KHND,2123.0,185.0,a3cc2a,2018-10-11 00:19:00,2018-10-11 00:42:21
2,WT77,6694.0,722.0,00WA,12319.0,1478.0,a8f0ab,2018-10-11 00:13:03,2018-10-11 00:24:15
3,KC83,6081.0,1682.0,KRHV,382.0,19.0,ab9659,2018-10-11 00:21:47,2018-10-11 00:34:27
4,KCCR,312.0,22.0,KOAK,1257.0,4.0,adc43a,2018-10-11 00:24:50,2018-10-11 00:37:27
5,KDCA,5362.0,254.0,KEWR,385.0,13.0,a9ffeb,2018-10-11 00:05:18,2018-10-11 00:47:11


In [17]:
NewData.describe()

,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,estDepartureAirportHorizDistance,estDepartureAirportVertDistance
count,237.000000,237.000000,237.000000,237.000000
mean,5954.970464,485.860759,4052.835443,441.004219
std,6198.960484,529.530814,4000.290231,479.460692
min,293.000000,4.000000,52.000000,2.000000
25%,1807.000000,87.000000,1104.000000,90.000000
50%,3562.000000,279.000000,2352.000000,258.000000
75%,7884.000000,728.000000,6091.000000,618.000000
max,29426.000000,2470.000000,14985.000000,1996.000000


In [19]:
# Get the difference between datetimes (as timedelta) lastSeen and firstSeen:

dateTimeDifference = NewData['lastSeenNew'] - NewData['firstSeenNew']
dateTimeDifference


0     00:23:21
2     00:11:12
3     00:12:40
4     00:12:37
5     00:41:53
6     00:29:47
8     00:36:24
10    00:14:43
11    00:20:34
13    00:18:14
19    00:28:20
20    00:18:30
21    00:23:49
23    00:20:59
24    00:26:51
25    00:22:36
26    00:20:02
28    00:27:50
29    00:10:11
30    00:31:04
32    00:17:32
35    00:14:14
36    00:14:17
37    00:24:41
38    00:38:40
39    00:15:24
41    00:32:21
44    00:27:41
46    00:17:39
47    00:14:20
        ...   
640   00:34:39
641   00:11:35
645   00:24:07
650   00:36:33
652   00:13:28
654   00:20:10
658   00:29:55
659   00:39:38
663   00:28:28
664   00:22:02
665   00:24:18
668   00:11:28
672   00:10:56
673   00:13:35
674   00:26:36
677   00:24:31
680   00:34:19
685   00:21:35
686   00:18:04
687   00:11:35
688   00:24:30
692   00:46:07
693   00:16:52
695   00:15:54
697   00:13:20
699   00:23:28
701   00:17:04
702   00:11:37
704   00:12:23
705   00:18:16
Length: 237, dtype: timedelta64[ns]

### Arrivals by airport :

In [20]:
# all flights arriving at Frankfurt International Airport (EDDF) from 12pm to 1pm on Jan 29 2018:

dd = pd.read_json('https://opensky-network.org/api/flights/arrival?airport=EDDF&begin=1517227200&end=1517230800')
dd.head(19)

,arrivalAirportCandidatesCount,callsign,departureAirportCandidatesCount,estArrivalAirport,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,estDepartureAirport,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,firstSeen,icao24,lastSeen
0,2,MSR785,0,EDDF,1593,95,None,NaN,NaN,1517220729,0101be,1517230737
1,3,DLH187,1,EDDF,3000,103,EDDT,191.0,54.0,1517227831,3c6675,1517230709
2,3,DLH8JT,0,EDDF,3307,126,None,NaN,NaN,1517226907,3c6663,1517230498
3,3,DLH543,0,EDDF,4478,126,None,NaN,NaN,1517196397,3c64c3,1517230408
4,2,ADR114,0,EDDF,357,95,None,NaN,NaN,1517222295,501d7d,1517230331
5,3,DLH1MA,0,EDDF,4044,133,LHBP,5462.0,801.0,1517225348,3c65c2,1517230259
6,2,SXS73C,0,EDDF,1543,103,None,NaN,NaN,1517220169,4bcdd5,1517230076
7,2,AZA408,1,EDDF,1322,103,LIML,1205.0,220.0,1517226616,4ca99c,1517229957
8,3,DLH3W,0,EDDF,4468,126,EBBR,603.0,86.0,1517227669,3c674e,1517229954
9,3,THY3JY,0,EDDF,1849,80,None,NaN,NaN,1517220062,4baa12,1517229677


In [21]:
dd.shape

(19, 12)

In [22]:
# changing the variable'firstSeen' to datetime formats :

from datetime import datetime

for index, row in dd.iterrows():   
    dt_object1 = datetime.fromtimestamp(row['firstSeen'])
    print("dt_object1 =", dt_object1)  

dt_object1 = 2018-01-29 11:12:09
dt_object1 = 2018-01-29 13:10:31
dt_object1 = 2018-01-29 12:55:07
dt_object1 = 2018-01-29 04:26:37
dt_object1 = 2018-01-29 11:38:15
dt_object1 = 2018-01-29 12:29:08
dt_object1 = 2018-01-29 11:02:49
dt_object1 = 2018-01-29 12:50:16
dt_object1 = 2018-01-29 13:07:49
dt_object1 = 2018-01-29 11:01:02
dt_object1 = 2018-01-29 12:55:41
dt_object1 = 2018-01-29 10:39:26
dt_object1 = 2018-01-29 11:53:01
dt_object1 = 2018-01-29 10:31:33
dt_object1 = 2018-01-29 12:27:21
dt_object1 = 2018-01-29 02:40:44
dt_object1 = 2018-01-29 12:18:32
dt_object1 = 2018-01-29 10:00:41
dt_object1 = 2018-01-29 11:50:09


In [23]:
# changing the variable'firstSeen' to datetime formats :

from datetime import datetime

for index, row in dd.iterrows():   
    dt_object2 = datetime.fromtimestamp(row['lastSeen'])
    print("dt_object2 =", dt_object2)  

dt_object2 = 2018-01-29 13:58:57
dt_object2 = 2018-01-29 13:58:29
dt_object2 = 2018-01-29 13:54:58
dt_object2 = 2018-01-29 13:53:28
dt_object2 = 2018-01-29 13:52:11
dt_object2 = 2018-01-29 13:50:59
dt_object2 = 2018-01-29 13:47:56
dt_object2 = 2018-01-29 13:45:57
dt_object2 = 2018-01-29 13:45:54
dt_object2 = 2018-01-29 13:41:17
dt_object2 = 2018-01-29 13:38:40
dt_object2 = 2018-01-29 13:31:44
dt_object2 = 2018-01-29 13:24:33
dt_object2 = 2018-01-29 13:21:15
dt_object2 = 2018-01-29 13:13:14
dt_object2 = 2018-01-29 13:09:26
dt_object2 = 2018-01-29 13:05:09
dt_object2 = 2018-01-29 13:04:55
dt_object2 = 2018-01-29 13:01:27


In [24]:
# adding a new column to our data containing the new values of 'firstSeen' :

from datetime import datetime

for index, row in dd.iterrows():
    dd['firstSeenNew1'] = dd.apply(lambda row: datetime.fromtimestamp(row['firstSeen']), axis=1) 

In [25]:
dd.head()

,arrivalAirportCandidatesCount,callsign,departureAirportCandidatesCount,estArrivalAirport,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,estDepartureAirport,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,firstSeen,icao24,lastSeen,firstSeenNew1
0,2,MSR785,0,EDDF,1593,95,None,NaN,NaN,1517220729,0101be,1517230737,2018-01-29 11:12:09
1,3,DLH187,1,EDDF,3000,103,EDDT,191.0,54.0,1517227831,3c6675,1517230709,2018-01-29 13:10:31
2,3,DLH8JT,0,EDDF,3307,126,None,NaN,NaN,1517226907,3c6663,1517230498,2018-01-29 12:55:07
3,3,DLH543,0,EDDF,4478,126,None,NaN,NaN,1517196397,3c64c3,1517230408,2018-01-29 04:26:37
4,2,ADR114,0,EDDF,357,95,None,NaN,NaN,1517222295,501d7d,1517230331,2018-01-29 11:38:15


In [26]:
# adding a new column to our data containing the new values of 'firstSeen' :

from datetime import datetime

for index, row in dd.iterrows():
    dd['lastSeenNew1'] = dd.apply(lambda row: datetime.fromtimestamp(row['lastSeen']), axis=1) 

In [27]:
dd.head(19)

,arrivalAirportCandidatesCount,callsign,departureAirportCandidatesCount,estArrivalAirport,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,estDepartureAirport,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,firstSeen,icao24,lastSeen,firstSeenNew1,lastSeenNew1
0,2,MSR785,0,EDDF,1593,95,None,NaN,NaN,1517220729,0101be,1517230737,2018-01-29 11:12:09,2018-01-29 13:58:57
1,3,DLH187,1,EDDF,3000,103,EDDT,191.0,54.0,1517227831,3c6675,1517230709,2018-01-29 13:10:31,2018-01-29 13:58:29
2,3,DLH8JT,0,EDDF,3307,126,None,NaN,NaN,1517226907,3c6663,1517230498,2018-01-29 12:55:07,2018-01-29 13:54:58
3,3,DLH543,0,EDDF,4478,126,None,NaN,NaN,1517196397,3c64c3,1517230408,2018-01-29 04:26:37,2018-01-29 13:53:28
4,2,ADR114,0,EDDF,357,95,None,NaN,NaN,1517222295,501d7d,1517230331,2018-01-29 11:38:15,2018-01-29 13:52:11
5,3,DLH1MA,0,EDDF,4044,133,LHBP,5462.0,801.0,1517225348,3c65c2,1517230259,2018-01-29 12:29:08,2018-01-29 13:50:59
6,2,SXS73C,0,EDDF,1543,103,None,NaN,NaN,1517220169,4bcdd5,1517230076,2018-01-29 11:02:49,2018-01-29 13:47:56
7,2,AZA408,1,EDDF,1322,103,LIML,1205.0,220.0,1517226616,4ca99c,1517229957,2018-01-29 12:50:16,2018-01-29 13:45:57
8,3,DLH3W,0,EDDF,4468,126,EBBR,603.0,86.0,1517227669,3c674e,1517229954,2018-01-29 13:07:49,2018-01-29 13:45:54
9,3,THY3JY,0,EDDF,1849,80,None,NaN,NaN,1517220062,4baa12,1517229677,2018-01-29 11:01:02,2018-01-29 13:41:17


In [28]:
# Get the difference between datetimes (as timedelta) lastSeen and firstSeen:

dateTimeDifference = dd['lastSeenNew1'] - dd['firstSeenNew1']
dateTimeDifference

0    02:46:48
1    00:47:58
2    00:59:51
3    09:26:51
4    02:13:56
5    01:21:51
6    02:45:07
7    00:55:41
8    00:38:05
9    02:40:15
10   00:42:59
11   02:52:18
12   01:31:32
13   02:49:42
14   00:45:53
15   10:28:42
16   00:46:37
17   03:04:14
18   01:11:18
dtype: timedelta64[ns]

In [29]:
# DELETE NULL VARIABLES :

dd1 = dd.dropna()
dd1.head()

,arrivalAirportCandidatesCount,callsign,departureAirportCandidatesCount,estArrivalAirport,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,estDepartureAirport,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,firstSeen,icao24,lastSeen,firstSeenNew1,lastSeenNew1
1,3,DLH187,1,EDDF,3000,103,EDDT,191.0,54.0,1517227831,3c6675,1517230709,2018-01-29 13:10:31,2018-01-29 13:58:29
5,3,DLH1MA,0,EDDF,4044,133,LHBP,5462.0,801.0,1517225348,3c65c2,1517230259,2018-01-29 12:29:08,2018-01-29 13:50:59
7,2,AZA408,1,EDDF,1322,103,LIML,1205.0,220.0,1517226616,4ca99c,1517229957,2018-01-29 12:50:16,2018-01-29 13:45:57
8,3,DLH3W,0,EDDF,4468,126,EBBR,603.0,86.0,1517227669,3c674e,1517229954,2018-01-29 13:07:49,2018-01-29 13:45:54
10,2,SJT911,0,EDDF,1576,88,LOWS,8534.0,1246.0,1517226941,4b19d1,1517229520,2018-01-29 12:55:41,2018-01-29 13:38:40


### Departures by airports 

In [30]:
# all flights departing at Frankfurt International Airport (EDDF) from 12pm to 1pm on Jan 29 2018:

aa = pd.read_json('https://opensky-network.org/api/flights/departure?airport=EDDF&begin=1517227200&end=1517230800')
aa.head()

,arrivalAirportCandidatesCount,callsign,departureAirportCandidatesCount,estArrivalAirport,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,estDepartureAirport,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,firstSeen,icao24,lastSeen
0,0,DLH630,1,None,NaN,NaN,EDDF,4319,65,1517230790,3c4ad0,1517238306
1,2,DLH5LA,1,LSGG,1803.0,163.0,EDDF,4362,103,1517230716,3c5469,1517233481
2,0,THY9AD,2,None,NaN,NaN,EDDF,3764,80,1517230625,4baa03,1517238702
3,3,TAP583,2,LPCS,14016.0,708.0,EDDF,3808,80,1517230550,4951d0,1517240237
4,0,CES220,1,None,NaN,NaN,EDDF,1464,11,1517230493,780b7d,1517295442


In [31]:
aa.shape

(50, 12)

In [32]:
tt = pd.read_json('https://opensky-network.org/api/flights/arrival?airport=DTTA&begin=1517227200&end=1517238000')
tt.shape

(14, 12)

In [33]:
tt.head(14)

,arrivalAirportCandidatesCount,callsign,departureAirportCandidatesCount,estArrivalAirport,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,estDepartureAirport,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,firstSeen,icao24,lastSeen
0,0,TAR791,1,DTTA,5108,336,EGKK,1178.0,167.0,1517228404,02a18f,1517237077
1,1,THY663,0,DTTA,9668,282,None,NaN,NaN,1517226724,4baa67,1517235205
2,1,LWA810,0,DTTA,9225,267,None,NaN,NaN,1517231403,018087,1517234022
3,0,LBT392,0,DTTA,11312,359,None,NaN,NaN,1517232198,02a1a1,1517232802
4,0,TAR717,1,DTTA,9134,237,LFPO,3369.0,12.0,1517224905,02a195,1517232432
5,0,TAR751,0,DTTA,11957,389,LFLL,12278.0,1974.0,1517225476,02a192,1517231507
6,0,TAR757,1,DTTA,10258,290,LIMC,432.0,142.0,1517225439,02a194,1517231000
7,0,TAR712,0,DTTA,9181,237,None,NaN,NaN,1517228523,02a191,1517230872
8,0,TAR745,2,DTTA,13636,366,EDDF,3956.0,80.0,1517222934,02a190,1517230462
9,0,RJA551,0,DTTA,19233,374,None,NaN,NaN,1517218486,740731,1517229886


In [34]:
# changing the variable'firstSeen' to datetime formats :

from datetime import datetime

for index, row in tt.iterrows():   
    dt_object1 = datetime.fromtimestamp(row['firstSeen'])
    print("dt_object1 =", dt_object1)

dt_object1 = 2018-01-29 13:20:04
dt_object1 = 2018-01-29 12:52:04
dt_object1 = 2018-01-29 14:10:03
dt_object1 = 2018-01-29 14:23:18
dt_object1 = 2018-01-29 12:21:45
dt_object1 = 2018-01-29 12:31:16
dt_object1 = 2018-01-29 12:30:39
dt_object1 = 2018-01-29 13:22:03
dt_object1 = 2018-01-29 11:48:54
dt_object1 = 2018-01-29 10:34:46
dt_object1 = 2018-01-29 11:32:31
dt_object1 = 2018-01-29 12:14:03
dt_object1 = 2018-01-29 11:23:33
dt_object1 = 2018-01-29 11:31:12


In [35]:
# changing the variable'lastSeen' to datetime formats :

from datetime import datetime

for index, row in tt.iterrows():   
    dt_object2 = datetime.fromtimestamp(row['lastSeen'])
    print("dt_object2 =", dt_object2)

dt_object2 = 2018-01-29 15:44:37
dt_object2 = 2018-01-29 15:13:25
dt_object2 = 2018-01-29 14:53:42
dt_object2 = 2018-01-29 14:33:22
dt_object2 = 2018-01-29 14:27:12
dt_object2 = 2018-01-29 14:11:47
dt_object2 = 2018-01-29 14:03:20
dt_object2 = 2018-01-29 14:01:12
dt_object2 = 2018-01-29 13:54:22
dt_object2 = 2018-01-29 13:44:46
dt_object2 = 2018-01-29 13:37:29
dt_object2 = 2018-01-29 13:32:55
dt_object2 = 2018-01-29 13:31:29
dt_object2 = 2018-01-29 13:26:47


In [36]:

from datetime import datetime

for index, row in tt.iterrows():
    tt['firstSeenNew2'] = tt.apply(lambda row: datetime.fromtimestamp(row['firstSeen']), axis=1) 

In [37]:
tt.head()

,arrivalAirportCandidatesCount,callsign,departureAirportCandidatesCount,estArrivalAirport,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,estDepartureAirport,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,firstSeen,icao24,lastSeen,firstSeenNew2
0,0,TAR791,1,DTTA,5108,336,EGKK,1178.0,167.0,1517228404,02a18f,1517237077,2018-01-29 13:20:04
1,1,THY663,0,DTTA,9668,282,None,NaN,NaN,1517226724,4baa67,1517235205,2018-01-29 12:52:04
2,1,LWA810,0,DTTA,9225,267,None,NaN,NaN,1517231403,018087,1517234022,2018-01-29 14:10:03
3,0,LBT392,0,DTTA,11312,359,None,NaN,NaN,1517232198,02a1a1,1517232802,2018-01-29 14:23:18
4,0,TAR717,1,DTTA,9134,237,LFPO,3369.0,12.0,1517224905,02a195,1517232432,2018-01-29 12:21:45


In [38]:

from datetime import datetime

for index, row in tt.iterrows():
    tt['lastSeenNew2'] = tt.apply(lambda row: datetime.fromtimestamp(row['lastSeen']), axis=1) 

In [39]:
tt.head(14)

,arrivalAirportCandidatesCount,callsign,departureAirportCandidatesCount,estArrivalAirport,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,estDepartureAirport,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,firstSeen,icao24,lastSeen,firstSeenNew2,lastSeenNew2
0,0,TAR791,1,DTTA,5108,336,EGKK,1178.0,167.0,1517228404,02a18f,1517237077,2018-01-29 13:20:04,2018-01-29 15:44:37
1,1,THY663,0,DTTA,9668,282,None,NaN,NaN,1517226724,4baa67,1517235205,2018-01-29 12:52:04,2018-01-29 15:13:25
2,1,LWA810,0,DTTA,9225,267,None,NaN,NaN,1517231403,018087,1517234022,2018-01-29 14:10:03,2018-01-29 14:53:42
3,0,LBT392,0,DTTA,11312,359,None,NaN,NaN,1517232198,02a1a1,1517232802,2018-01-29 14:23:18,2018-01-29 14:33:22
4,0,TAR717,1,DTTA,9134,237,LFPO,3369.0,12.0,1517224905,02a195,1517232432,2018-01-29 12:21:45,2018-01-29 14:27:12
5,0,TAR751,0,DTTA,11957,389,LFLL,12278.0,1974.0,1517225476,02a192,1517231507,2018-01-29 12:31:16,2018-01-29 14:11:47
6,0,TAR757,1,DTTA,10258,290,LIMC,432.0,142.0,1517225439,02a194,1517231000,2018-01-29 12:30:39,2018-01-29 14:03:20
7,0,TAR712,0,DTTA,9181,237,None,NaN,NaN,1517228523,02a191,1517230872,2018-01-29 13:22:03,2018-01-29 14:01:12
8,0,TAR745,2,DTTA,13636,366,EDDF,3956.0,80.0,1517222934,02a190,1517230462,2018-01-29 11:48:54,2018-01-29 13:54:22
9,0,RJA551,0,DTTA,19233,374,None,NaN,NaN,1517218486,740731,1517229886,2018-01-29 10:34:46,2018-01-29 13:44:46


In [40]:
tt1 = tt.dropna()
tt1

,arrivalAirportCandidatesCount,callsign,departureAirportCandidatesCount,estArrivalAirport,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,estDepartureAirport,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,firstSeen,icao24,lastSeen,firstSeenNew2,lastSeenNew2
0,0,TAR791,1,DTTA,5108,336,EGKK,1178.0,167.0,1517228404,02a18f,1517237077,2018-01-29 13:20:04,2018-01-29 15:44:37
4,0,TAR717,1,DTTA,9134,237,LFPO,3369.0,12.0,1517224905,02a195,1517232432,2018-01-29 12:21:45,2018-01-29 14:27:12
5,0,TAR751,0,DTTA,11957,389,LFLL,12278.0,1974.0,1517225476,02a192,1517231507,2018-01-29 12:31:16,2018-01-29 14:11:47
6,0,TAR757,1,DTTA,10258,290,LIMC,432.0,142.0,1517225439,02a194,1517231000,2018-01-29 12:30:39,2018-01-29 14:03:20
8,0,TAR745,2,DTTA,13636,366,EDDF,3956.0,80.0,1517222934,02a190,1517230462,2018-01-29 11:48:54,2018-01-29 13:54:22
11,0,TAR363,0,DTTA,18284,488,LIPE,8914.0,1288.0,1517224443,02a1cb,1517229175,2018-01-29 12:14:03,2018-01-29 13:32:55
12,0,LBT315,1,DTTA,12305,374,LFPG,2095.0,20.0,1517221413,02a1af,1517229089,2018-01-29 11:23:33,2018-01-29 13:31:29


In [41]:
# Get the difference between datetimes (as timedelta) lastSeen and firstSeen:

dateTimeDifference = tt1['lastSeenNew2'] - tt1['firstSeenNew2']
dateTimeDifference

0    02:24:33
4    02:05:27
5    01:40:31
6    01:32:41
8    02:05:28
11   01:18:52
12   02:07:56
dtype: timedelta64[ns]

In [42]:
#"https://opensky-network.org/api/flights/arrival?airport=DTTA&begin=1517227200&end=1517230800"
# importing the data from Tunis airport: 

### imports the library requests :
import requests

### making a request to API OpenSky Network and check its status :
response = requests.get('https://opensky-network.org/api/flights/arrival?airport=DTTA&begin=1517227200&end=1517270400')
print(response.status_code)

200


In [43]:
#calculating the distance from departure airport and from the arrival airport :

import pandas as pd

df=pd.read_json('https://opensky-network.org/api/flights/arrival?airport=DTTA&begin=1517227200&end=1517270400')


import pandas as pd
import numpy as np
list=[]
list2=[]
list3=[]

for i in range(len(df)):    
    dep = np.sqrt(np.square(df['estArrivalAirportHorizDistance'][i]+np.square(df['estArrivalAirportVertDistance'][i])))
    arr = np.sqrt(np.square(df['estDepartureAirportHorizDistance'][i]+np.square(df['estDepartureAirportVertDistance'][i])))
    pr = dep/(dep+arr)
    list2.append(arr)
    list.append(dep)
    list3.append(np.round(pr*100,decimals=2))
    



df['Distance from Departure Airport']=list
df['Distance left to the Arrival Airport']=list2
df['Progress in %']=list3





In [44]:
# Dropping the NaN values in the data :

df = df.dropna()
df.head()

,arrivalAirportCandidatesCount,callsign,departureAirportCandidatesCount,estArrivalAirport,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,estDepartureAirport,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,firstSeen,icao24,lastSeen,Distance from Departure Airport,Distance left to the Arrival Airport,Progress in %
0,0,DLH1326,2,DTTA,7446,145,EDDF,3884.0,49.0,1517261275,3c66aa,1517268546,28471.0,6285.0,81.92
4,0,TAR723,1,DTTA,7137,115,LFPO,3552.0,27.0,1517251730,02a18b,1517258882,20362.0,4281.0,82.63
5,0,TAR319,2,DTTA,7032,115,LPPT,4323.0,952.0,1517249293,02a1cb,1517257303,20257.0,910627.0,2.18
6,0,TAR629,2,DTTA,8721,145,LFBO,770.0,136.0,1517251366,02a191,1517257061,29746.0,19266.0,60.69
7,0,LBT357,2,DTTA,7135,138,LFBO,1867.0,30.0,1517249941,02a1af,1517255638,26179.0,2767.0,90.44


In [45]:
print('Dataframe dimensions:', df.shape)

Dataframe dimensions: (23, 15)


In [46]:
# changing the variable'firstSeen' to datetime formats :

from datetime import datetime

for index, row in df.iterrows():   
    dt_object1 = datetime.fromtimestamp(row['firstSeen'])
    print("dt_object1 =", dt_object1)

dt_object1 = 2018-01-29 22:27:55
dt_object1 = 2018-01-29 19:48:50
dt_object1 = 2018-01-29 19:08:13
dt_object1 = 2018-01-29 19:42:46
dt_object1 = 2018-01-29 19:19:01
dt_object1 = 2018-01-29 18:24:01
dt_object1 = 2018-01-29 18:58:10
dt_object1 = 2018-01-29 17:09:05
dt_object1 = 2018-01-29 17:04:39
dt_object1 = 2018-01-29 17:11:10
dt_object1 = 2018-01-29 16:48:35
dt_object1 = 2018-01-29 16:51:39
dt_object1 = 2018-01-29 16:11:38
dt_object1 = 2018-01-29 16:59:05
dt_object1 = 2018-01-29 17:00:12
dt_object1 = 2018-01-29 14:27:33
dt_object1 = 2018-01-29 13:20:04
dt_object1 = 2018-01-29 12:21:45
dt_object1 = 2018-01-29 12:31:16
dt_object1 = 2018-01-29 12:30:39
dt_object1 = 2018-01-29 11:48:54
dt_object1 = 2018-01-29 12:14:03
dt_object1 = 2018-01-29 11:23:33


In [47]:
# changing the variable'LastSeen' to datetime formats :

from datetime import datetime

for index, row in df.iterrows():   
    dt_object1 = datetime.fromtimestamp(row['lastSeen'])
    print("dt_object1 =", dt_object1)

dt_object1 = 2018-01-30 00:29:06
dt_object1 = 2018-01-29 21:48:02
dt_object1 = 2018-01-29 21:21:43
dt_object1 = 2018-01-29 21:17:41
dt_object1 = 2018-01-29 20:53:58
dt_object1 = 2018-01-29 20:50:37
dt_object1 = 2018-01-29 19:35:48
dt_object1 = 2018-01-29 19:25:05
dt_object1 = 2018-01-29 19:18:55
dt_object1 = 2018-01-29 18:55:15
dt_object1 = 2018-01-29 18:46:12
dt_object1 = 2018-01-29 18:31:57
dt_object1 = 2018-01-29 18:08:50
dt_object1 = 2018-01-29 17:52:54
dt_object1 = 2018-01-29 17:47:15
dt_object1 = 2018-01-29 16:39:58
dt_object1 = 2018-01-29 15:44:37
dt_object1 = 2018-01-29 14:27:12
dt_object1 = 2018-01-29 14:11:47
dt_object1 = 2018-01-29 14:03:20
dt_object1 = 2018-01-29 13:54:22
dt_object1 = 2018-01-29 13:32:55
dt_object1 = 2018-01-29 13:31:29


In [49]:
# adding two new columns to our data containing the new values of 'firstSeen' and 'lastSeen' :

from datetime import datetime

for index, row in df.iterrows():
    df['firstSeenNew'] = df.apply(lambda row: datetime.fromtimestamp(row['firstSeen']), axis=1)
    df['lastSeenNew'] = df.apply(lambda row: datetime.fromtimestamp(row['lastSeen']), axis=1) 
df.head()    

,arrivalAirportCandidatesCount,callsign,departureAirportCandidatesCount,estArrivalAirport,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,estDepartureAirport,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,firstSeen,icao24,lastSeen,Distance from Departure Airport,Distance left to the Arrival Airport,Progress in %,firstSeenNew,lastSeenNew
0,0,DLH1326,2,DTTA,7446,145,EDDF,3884.0,49.0,1517261275,3c66aa,1517268546,28471.0,6285.0,81.92,2018-01-29 22:27:55,2018-01-30 00:29:06
4,0,TAR723,1,DTTA,7137,115,LFPO,3552.0,27.0,1517251730,02a18b,1517258882,20362.0,4281.0,82.63,2018-01-29 19:48:50,2018-01-29 21:48:02
5,0,TAR319,2,DTTA,7032,115,LPPT,4323.0,952.0,1517249293,02a1cb,1517257303,20257.0,910627.0,2.18,2018-01-29 19:08:13,2018-01-29 21:21:43
6,0,TAR629,2,DTTA,8721,145,LFBO,770.0,136.0,1517251366,02a191,1517257061,29746.0,19266.0,60.69,2018-01-29 19:42:46,2018-01-29 21:17:41
7,0,LBT357,2,DTTA,7135,138,LFBO,1867.0,30.0,1517249941,02a1af,1517255638,26179.0,2767.0,90.44,2018-01-29 19:19:01,2018-01-29 20:53:58


In [51]:
# calculating the flight duration 

for index, row in df.iterrows():
    df['duree_du_vol'] = df.apply(lambda row: (row['lastSeenNew'] - row['firstSeenNew']), axis=1)

In [53]:
result=[]
#i=0
for index, row in df.iterrows():
    t = str(row['duree_du_vol'])
    p = t[7:len(t)]
    (h,m,s) = p.split(':')
    result.append((int(h) * 3600 + int(m) * 60 + int(s))/3600)
df['duree_en_heures'] = result
df.head(5)
    

,arrivalAirportCandidatesCount,callsign,departureAirportCandidatesCount,estArrivalAirport,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,estDepartureAirport,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,firstSeen,icao24,lastSeen,Distance from Departure Airport,Distance left to the Arrival Airport,Progress in %,firstSeenNew,lastSeenNew,duree_du_vol,duree_en_heures
0,0,DLH1326,2,DTTA,7446,145,EDDF,3884.0,49.0,1517261275,3c66aa,1517268546,28471.0,6285.0,81.92,2018-01-29 22:27:55,2018-01-30 00:29:06,02:01:11,2.019722
4,0,TAR723,1,DTTA,7137,115,LFPO,3552.0,27.0,1517251730,02a18b,1517258882,20362.0,4281.0,82.63,2018-01-29 19:48:50,2018-01-29 21:48:02,01:59:12,1.986667
5,0,TAR319,2,DTTA,7032,115,LPPT,4323.0,952.0,1517249293,02a1cb,1517257303,20257.0,910627.0,2.18,2018-01-29 19:08:13,2018-01-29 21:21:43,02:13:30,2.225000
6,0,TAR629,2,DTTA,8721,145,LFBO,770.0,136.0,1517251366,02a191,1517257061,29746.0,19266.0,60.69,2018-01-29 19:42:46,2018-01-29 21:17:41,01:34:55,1.581944
7,0,LBT357,2,DTTA,7135,138,LFBO,1867.0,30.0,1517249941,02a1af,1517255638,26179.0,2767.0,90.44,2018-01-29 19:19:01,2018-01-29 20:53:58,01:34:57,1.582500


In [55]:
# Calculating the elapsed time

result=[]

for index, row in df.iterrows():
    vm = row['duree_en_heures'] * (row['Progress in %']/100)
    result.append(vm)
df['temps_ecoule'] = result
df.head(5)

,arrivalAirportCandidatesCount,callsign,departureAirportCandidatesCount,estArrivalAirport,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,estDepartureAirport,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,firstSeen,icao24,lastSeen,Distance from Departure Airport,Distance left to the Arrival Airport,Progress in %,firstSeenNew,lastSeenNew,duree_du_vol,duree_en_heures,temps_ecoule
0,0,DLH1326,2,DTTA,7446,145,EDDF,3884.0,49.0,1517261275,3c66aa,1517268546,28471.0,6285.0,81.92,2018-01-29 22:27:55,2018-01-30 00:29:06,02:01:11,2.019722,1.654556
4,0,TAR723,1,DTTA,7137,115,LFPO,3552.0,27.0,1517251730,02a18b,1517258882,20362.0,4281.0,82.63,2018-01-29 19:48:50,2018-01-29 21:48:02,01:59:12,1.986667,1.641583
5,0,TAR319,2,DTTA,7032,115,LPPT,4323.0,952.0,1517249293,02a1cb,1517257303,20257.0,910627.0,2.18,2018-01-29 19:08:13,2018-01-29 21:21:43,02:13:30,2.225000,0.048505
6,0,TAR629,2,DTTA,8721,145,LFBO,770.0,136.0,1517251366,02a191,1517257061,29746.0,19266.0,60.69,2018-01-29 19:42:46,2018-01-29 21:17:41,01:34:55,1.581944,0.960082
7,0,LBT357,2,DTTA,7135,138,LFBO,1867.0,30.0,1517249941,02a1af,1517255638,26179.0,2767.0,90.44,2018-01-29 19:19:01,2018-01-29 20:53:58,01:34:57,1.582500,1.431213


In [56]:
# Average value since departure 

result=[]
for index ,row in df.iterrows():
    vm = vm = row['Distance from Departure Airport']/row['temps_ecoule']
    result.append(vm)
df['VM_depuis_le_depart'] = result
df.head(5)

,arrivalAirportCandidatesCount,callsign,departureAirportCandidatesCount,estArrivalAirport,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,estDepartureAirport,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,firstSeen,...,lastSeen,Distance from Departure Airport,Distance left to the Arrival Airport,Progress in %,firstSeenNew,lastSeenNew,duree_du_vol,duree_en_heures,temps_ecoule,VM_depuis_le_depart
0,0,DLH1326,2,DTTA,7446,145,EDDF,3884.0,49.0,1517261275,...,1517268546,28471.0,6285.0,81.92,2018-01-29 22:27:55,2018-01-30 00:29:06,02:01:11,2.019722,1.654556,17207.632955
4,0,TAR723,1,DTTA,7137,115,LFPO,3552.0,27.0,1517251730,...,1517258882,20362.0,4281.0,82.63,2018-01-29 19:48:50,2018-01-29 21:48:02,01:59:12,1.986667,1.641583,12403.883407
5,0,TAR319,2,DTTA,7032,115,LPPT,4323.0,952.0,1517249293,...,1517257303,20257.0,910627.0,2.18,2018-01-29 19:08:13,2018-01-29 21:21:43,02:13:30,2.225000,0.048505,417627.048758
6,0,TAR629,2,DTTA,8721,145,LFBO,770.0,136.0,1517251366,...,1517257061,29746.0,19266.0,60.69,2018-01-29 19:42:46,2018-01-29 21:17:41,01:34:55,1.581944,0.960082,30982.767532
7,0,LBT357,2,DTTA,7135,138,LFBO,1867.0,30.0,1517249941,...,1517255638,26179.0,2767.0,90.44,2018-01-29 19:19:01,2018-01-29 20:53:58,01:34:57,1.582500,1.431213,18291.477229


In [57]:
result=[]
for index ,row in df.iterrows():
    t = row['duree_en_heures'] - row['temps_ecoule']
    result.append(t)
df['timeleft'] = result
df.head()

,arrivalAirportCandidatesCount,callsign,departureAirportCandidatesCount,estArrivalAirport,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,estDepartureAirport,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,firstSeen,...,Distance from Departure Airport,Distance left to the Arrival Airport,Progress in %,firstSeenNew,lastSeenNew,duree_du_vol,duree_en_heures,temps_ecoule,VM_depuis_le_depart,timeleft
0,0,DLH1326,2,DTTA,7446,145,EDDF,3884.0,49.0,1517261275,...,28471.0,6285.0,81.92,2018-01-29 22:27:55,2018-01-30 00:29:06,02:01:11,2.019722,1.654556,17207.632955,0.365166
4,0,TAR723,1,DTTA,7137,115,LFPO,3552.0,27.0,1517251730,...,20362.0,4281.0,82.63,2018-01-29 19:48:50,2018-01-29 21:48:02,01:59:12,1.986667,1.641583,12403.883407,0.345084
5,0,TAR319,2,DTTA,7032,115,LPPT,4323.0,952.0,1517249293,...,20257.0,910627.0,2.18,2018-01-29 19:08:13,2018-01-29 21:21:43,02:13:30,2.225000,0.048505,417627.048758,2.176495
6,0,TAR629,2,DTTA,8721,145,LFBO,770.0,136.0,1517251366,...,29746.0,19266.0,60.69,2018-01-29 19:42:46,2018-01-29 21:17:41,01:34:55,1.581944,0.960082,30982.767532,0.621862
7,0,LBT357,2,DTTA,7135,138,LFBO,1867.0,30.0,1517249941,...,26179.0,2767.0,90.44,2018-01-29 19:19:01,2018-01-29 20:53:58,01:34:57,1.582500,1.431213,18291.477229,0.151287


In [58]:
result=[]
for index ,row in df.iterrows():
    d= row['Distance from Departure Airport']*row['timeleft']
    result.append(d)
df['Projected_theorical_distance_left'] = result
df.head()

,arrivalAirportCandidatesCount,callsign,departureAirportCandidatesCount,estArrivalAirport,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,estDepartureAirport,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,firstSeen,...,Distance left to the Arrival Airport,Progress in %,firstSeenNew,lastSeenNew,duree_du_vol,duree_en_heures,temps_ecoule,VM_depuis_le_depart,timeleft,Projected_theorical_distance_left
0,0,DLH1326,2,DTTA,7446,145,EDDF,3884.0,49.0,1517261275,...,6285.0,81.92,2018-01-29 22:27:55,2018-01-30 00:29:06,02:01:11,2.019722,1.654556,17207.632955,0.365166,10396.634859
4,0,TAR723,1,DTTA,7137,115,LFPO,3552.0,27.0,1517251730,...,4281.0,82.63,2018-01-29 19:48:50,2018-01-29 21:48:02,01:59:12,1.986667,1.641583,12403.883407,0.345084,7026.600408
5,0,TAR319,2,DTTA,7032,115,LPPT,4323.0,952.0,1517249293,...,910627.0,2.18,2018-01-29 19:08:13,2018-01-29 21:21:43,02:13:30,2.225000,0.048505,417627.048758,2.176495,44089.259215
6,0,TAR629,2,DTTA,8721,145,LFBO,770.0,136.0,1517251366,...,19266.0,60.69,2018-01-29 19:42:46,2018-01-29 21:17:41,01:34:55,1.581944,0.960082,30982.767532,0.621862,18497.917794
7,0,LBT357,2,DTTA,7135,138,LFBO,1867.0,30.0,1517249941,...,2767.0,90.44,2018-01-29 19:19:01,2018-01-29 20:53:58,01:34:57,1.582500,1.431213,18291.477229,0.151287,3960.542373


In [60]:
result=[]
for index ,row in df.iterrows():
    p= row['Projected_theorical_distance_left']/row['Distance left to the Arrival Airport']
    result.append(p)
df['Punctuality Score'] = result
df.head()

,arrivalAirportCandidatesCount,callsign,departureAirportCandidatesCount,estArrivalAirport,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,estDepartureAirport,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,firstSeen,...,Progress in %,firstSeenNew,lastSeenNew,duree_du_vol,duree_en_heures,temps_ecoule,VM_depuis_le_depart,timeleft,Projected_theorical_distance_left,Punctuality Score
0,0,DLH1326,2,DTTA,7446,145,EDDF,3884.0,49.0,1517261275,...,81.92,2018-01-29 22:27:55,2018-01-30 00:29:06,02:01:11,2.019722,1.654556,17207.632955,0.365166,10396.634859,1.654198
4,0,TAR723,1,DTTA,7137,115,LFPO,3552.0,27.0,1517251730,...,82.63,2018-01-29 19:48:50,2018-01-29 21:48:02,01:59:12,1.986667,1.641583,12403.883407,0.345084,7026.600408,1.641346
5,0,TAR319,2,DTTA,7032,115,LPPT,4323.0,952.0,1517249293,...,2.18,2018-01-29 19:08:13,2018-01-29 21:21:43,02:13:30,2.225000,0.048505,417627.048758,2.176495,44089.259215,0.048416
6,0,TAR629,2,DTTA,8721,145,LFBO,770.0,136.0,1517251366,...,60.69,2018-01-29 19:42:46,2018-01-29 21:17:41,01:34:55,1.581944,0.960082,30982.767532,0.621862,18497.917794,0.960133
7,0,LBT357,2,DTTA,7135,138,LFBO,1867.0,30.0,1517249941,...,90.44,2018-01-29 19:19:01,2018-01-29 20:53:58,01:34:57,1.582500,1.431213,18291.477229,0.151287,3960.542373,1.431349


In [61]:
result=[]
for index ,row in df.iterrows():
    if row['Punctuality Score'] > 1:
        result.append('SLOW DOWN !')
    else:
        result.append('SPEED UP!')
df['Order'] = result
df.head()

,arrivalAirportCandidatesCount,callsign,departureAirportCandidatesCount,estArrivalAirport,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,estDepartureAirport,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,firstSeen,...,firstSeenNew,lastSeenNew,duree_du_vol,duree_en_heures,temps_ecoule,VM_depuis_le_depart,timeleft,Projected_theorical_distance_left,Punctuality Score,Order
0,0,DLH1326,2,DTTA,7446,145,EDDF,3884.0,49.0,1517261275,...,2018-01-29 22:27:55,2018-01-30 00:29:06,02:01:11,2.019722,1.654556,17207.632955,0.365166,10396.634859,1.654198,SLOW DOWN !
4,0,TAR723,1,DTTA,7137,115,LFPO,3552.0,27.0,1517251730,...,2018-01-29 19:48:50,2018-01-29 21:48:02,01:59:12,1.986667,1.641583,12403.883407,0.345084,7026.600408,1.641346,SLOW DOWN !
5,0,TAR319,2,DTTA,7032,115,LPPT,4323.0,952.0,1517249293,...,2018-01-29 19:08:13,2018-01-29 21:21:43,02:13:30,2.225000,0.048505,417627.048758,2.176495,44089.259215,0.048416,SPEED UP!
6,0,TAR629,2,DTTA,8721,145,LFBO,770.0,136.0,1517251366,...,2018-01-29 19:42:46,2018-01-29 21:17:41,01:34:55,1.581944,0.960082,30982.767532,0.621862,18497.917794,0.960133,SPEED UP!
7,0,LBT357,2,DTTA,7135,138,LFBO,1867.0,30.0,1517249941,...,2018-01-29 19:19:01,2018-01-29 20:53:58,01:34:57,1.582500,1.431213,18291.477229,0.151287,3960.542373,1.431349,SLOW DOWN !


In [63]:
result=[]
for index ,row in df.iterrows():
    p= row['Distance left to the Arrival Airport'] / row['timeleft']
    result.append(round(p))
df['Recommended Average Speed'] = result
df.head()

,arrivalAirportCandidatesCount,callsign,departureAirportCandidatesCount,estArrivalAirport,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,estDepartureAirport,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,firstSeen,...,lastSeenNew,duree_du_vol,duree_en_heures,temps_ecoule,VM_depuis_le_depart,timeleft,Projected_theorical_distance_left,Punctuality Score,Order,Recommended Average Speed
0,0,DLH1326,2,DTTA,7446,145,EDDF,3884.0,49.0,1517261275,...,2018-01-30 00:29:06,02:01:11,2.019722,1.654556,17207.632955,0.365166,10396.634859,1.654198,SLOW DOWN !,17211
4,0,TAR723,1,DTTA,7137,115,LFPO,3552.0,27.0,1517251730,...,2018-01-29 21:48:02,01:59:12,1.986667,1.641583,12403.883407,0.345084,7026.600408,1.641346,SLOW DOWN !,12406
5,0,TAR319,2,DTTA,7032,115,LPPT,4323.0,952.0,1517249293,...,2018-01-29 21:21:43,02:13:30,2.225000,0.048505,417627.048758,2.176495,44089.259215,0.048416,SPEED UP!,418391
6,0,TAR629,2,DTTA,8721,145,LFBO,770.0,136.0,1517251366,...,2018-01-29 21:17:41,01:34:55,1.581944,0.960082,30982.767532,0.621862,18497.917794,0.960133,SPEED UP!,30981
7,0,LBT357,2,DTTA,7135,138,LFBO,1867.0,30.0,1517249941,...,2018-01-29 20:53:58,01:34:57,1.582500,1.431213,18291.477229,0.151287,3960.542373,1.431349,SLOW DOWN !,18290
